In [1]:
from curses import init_color
import os
import os.path as osp
import copy
import sys
import time 
from datetime import timedelta
from datetime import datetime
import random
import time
from tqdm import tqdm
from collections import Counter

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import argparse
import pickle
from sklearn.metrics import f1_score, accuracy_score, confusion_matrix, classification_report

import torch
import torch.nn as nn
from torch.optim import lr_scheduler
from torch.utils.tensorboard import SummaryWriter
from torch.utils.data import DataLoader

from models.AutoEncoder import AutoEncoder_RGB, AutoEncoder_Depth
from models.AutoEncoder import AutoEncoder_Intergrated_Basic, AutoEncoder_Intergrated_Proposed
from models.hm_Network import Face_Detection_Model
# from models.Network import Face_Detection_Model

from dataloader.dataloader import load_dataset, load_test_dataset
from utility import draw_train_and_test_loss, draw_train_and_test_loss1, draw_accuracy_and_f1_during_training, draw_accuracy_and_f1_during_training1

print(torch.__version__,torch.cuda.is_available(),torch.cuda.device_count())
copy_before_model = True
server = 'server1'
print(f'server : {server} , copy_before_model : {copy_before_model}')

1.10.1+cu102 True 2
server : server1 , copy_before_model : True


In [2]:
# rgbp = 6
model = Face_Detection_Model(6).to('cuda')   

In [3]:
model2 = Face_Detection_Model(3).to('cuda')   

In [4]:
model = Face_Detection_Model(6).to('cuda')   

del model.bn1
del model.layer1[0].bn1
del model.layer1[0].bn2
del model.layer1[1].bn1
del model.layer1[1].bn2
del model.layer1[2].bn1
del model.layer1[2].bn2

del model.layer2[0].bn1
del model.layer2[0].bn2
del model.layer2[0].downsample[1]
del model.layer2[1].bn1
del model.layer2[1].bn2
del model.layer2[2].bn1
del model.layer2[2].bn2
del model.layer2[3].bn1
del model.layer2[3].bn2

del model.layer3[0].bn1
del model.layer3[0].bn2
del model.layer3[0].downsample[1]
del model.layer3[1].bn1
del model.layer3[1].bn2
del model.layer3[2].bn1
del model.layer3[2].bn2
del model.layer3[3].bn1
del model.layer3[3].bn2
del model.layer3[4].bn1
del model.layer3[4].bn2
del model.layer3[5].bn1
del model.layer3[5].bn2

del model.layer4[0].bn1
del model.layer4[0].bn2
del model.layer4[0].downsample[1]
del model.layer4[1].bn1
del model.layer4[1].bn2
del model.layer4[2].bn1
del model.layer4[2].bn2

In [6]:
from torchsummary import summary


In [7]:
summary(model, (6, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 14, 14]          18,816
              ReLU-2           [-1, 64, 14, 14]               0
         MaxPool2d-3             [-1, 64, 7, 7]               0
            Conv2d-4             [-1, 64, 7, 7]          36,864
              ReLU-5             [-1, 64, 7, 7]               0
            Conv2d-6             [-1, 64, 7, 7]          36,864
              ReLU-7             [-1, 64, 7, 7]               0
        BasicBlock-8             [-1, 64, 7, 7]               0
            Conv2d-9             [-1, 64, 7, 7]          36,864
             ReLU-10             [-1, 64, 7, 7]               0
           Conv2d-11             [-1, 64, 7, 7]          36,864
             ReLU-12             [-1, 64, 7, 7]               0
       BasicBlock-13             [-1, 64, 7, 7]               0
           Conv2d-14             [-1, 6

In [8]:
summary(model2, (3, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 14, 14]           9,408
       BatchNorm2d-2           [-1, 64, 14, 14]             128
              ReLU-3           [-1, 64, 14, 14]               0
         MaxPool2d-4             [-1, 64, 7, 7]               0
            Conv2d-5             [-1, 64, 7, 7]          36,864
       BatchNorm2d-6             [-1, 64, 7, 7]             128
              ReLU-7             [-1, 64, 7, 7]               0
            Conv2d-8             [-1, 64, 7, 7]          36,864
       BatchNorm2d-9             [-1, 64, 7, 7]             128
             ReLU-10             [-1, 64, 7, 7]               0
       BasicBlock-11             [-1, 64, 7, 7]               0
           Conv2d-12             [-1, 64, 7, 7]          36,864
      BatchNorm2d-13             [-1, 64, 7, 7]             128
             ReLU-14             [-1, 6